# Final Project: National Economic Accounts Comparison

This project creates a database for a worlwdide comparision of national income account-type variables converted to international prices. The homogenization of national accounts to a common numeraire allows for accurate comparisons of income between countries. Our database utilizes various comparable statistics to asses these differences. For the purpose of our project, we drew on data sets related to levels of education by gender and the ease of doing business in a country. We believe the information derived from these data sets provide meaningful insights into the economic realities of the nations examined. (ex: How levels of educational attainment, highest level of education completed, relate to household consumption. However, that is out-of-scope for now!).

__The dataset sources are WRDS and UNData.Org__

There is also a myexhangerateapiutil that has 2 functions - one gets all the currencynames available, and the second provides historical exchange rates for a currency to base currncy (USD). 

__Finally, use comments to document what you are doing with each block of code. Poorly documented code will be penalized accordingly.__

## 1. Import/Load all required packages.

In [107]:
import sqlite3
import pandas as pd
import csv
from  myexchangerateapiutil import get_Currencies

%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## 2. Create a SQLite database named 'ProjectNationalAccounts.db'. 

In [109]:
%sql sqlite:///ProjectNationalAccounts.db

'Connected: None@ProjectNationalAccounts.db'

## 3. Study the source data and design your database.

## 4. Assemble the data into lists and/or DataFrames

In [89]:
currList = []

currList.append(get_Currencies())
Currency = pd.DataFrame(currList).melt()

Currency.head(5)

,variable,value
0,AED,United Arab Emirates Dirham
1,AFN,Afghan Afghani
2,ALL,Albanian Lek
3,AMD,Armenian Dram
4,ANG,Netherlands Antillean Guilder


In [101]:
df1 = pd.read_csv('NationalAccounts.csv',header=0)
df2 = pd.read_csv('EaseofBusiness.csv',header=0)
df3 = pd.read_csv('Education.csv',header=0)
list_ = []
list_.append(df2[['AttCode','AttName']])

result = pd.merge(df2,df3,how='outer')
indi = pd.merge(result[['AttCode','AttName']].drop_duplicates(),df3[['AttCode','AttName']].drop_duplicates(),how='outer')

indicators = pd.merge(indi,df1[['AttCode','AttName']].drop_duplicates(),how='outer')

indicators.rename(columns={'AttCode':'IndCode','AttName':'IndName'},inplace=True)
indicators

,IndCode,IndName
0,IC.REG.DURS,Time required to start a business (days)
1,IC.TAX.TOTL.CP.ZS,Total tax rate (% of profit)
2,IC.REG.COST.PC.ZS,Cost to start a business (% of income per capita)
3,IC.RP.COST,Cost to register property (% of property value)
4,SE.PRM.TENR,"Adjusted net enrolment rate, primary, both sex..."
5,UIS.LP.AG15T99,"Adult illiterate population, 15+ years, both s..."
6,SE.ADT.LITR.ZS,"Adult literacy rate, population 15+ years, bot..."
7,SE.TER.CMPL.ZS,Gross graduation ratio from first degree progr...
8,POP,Population
9,CHCUR,Final Consumption


In [87]:
df1 = pd.read_csv('NationalAccounts.csv',header=0)
df2 = pd.read_csv('EaseofBusiness.csv',header=0)
df3 = pd.read_csv('Education.csv',header=0)
a = pd.merge(df2[['CntryCo','Cntry']].drop_duplicates(),df3[['CntryCo','Cntry']].drop_duplicates(),how='outer')
country = pd.merge(a,df1[['CntryCo','Cntry']].drop_duplicates(),how='outer')

country.head(5)

,CntryCo,Cntry
0,AFG,Afghanistan
1,ALB,Albania
2,DZA,Algeria
3,ASM,American Samoa
4,AND,Andorra


In [105]:
x = pd.merge(df2[['CntryCo','Year','AttCode','AttValue']],df3[['CntryCo','Year','AttCode','AttValue']],how='outer')
NationalAccounts = pd.merge(x,df1[['CntryCo','Year','AttCode','AttValue']],how='outer')

NationalAccounts.rename(columns={'AttCode':'IndCode','AttValue':'IndValue'},inplace=True)
NationalAccounts.head(15)
#x.drop_duplicates()

,CntryCo,Year,IndCode,IndValue
0,AFG,2007,IC.REG.DURS,9
1,AFG,2008,IC.REG.DURS,9
2,AFG,2009,IC.REG.DURS,7
3,AFG,2007,IC.TAX.TOTL.CP.ZS,36.8
4,AFG,2008,IC.TAX.TOTL.CP.ZS,36.8
5,AFG,2009,IC.TAX.TOTL.CP.ZS,36.8
6,AFG,2007,IC.REG.COST.PC.ZS,84.6
7,AFG,2008,IC.REG.COST.PC.ZS,59.5
8,AFG,2009,IC.REG.COST.PC.ZS,30.2
9,AFG,2007,IC.RP.COST,7


In [110]:
%%sql
DROP TABLE IF EXISTS CURRENCY;
DROP TABLE IF EXISTS INDICATORS;
DROP TABLE IF EXISTS COUNTRY;
DROP TABLE IF EXISTS NATIONALACCOUNTS;

CREATE TABLE CURRENCY (CurrCode TEXT PRIMARY KEY,CurrName TEXT NOT NULL);
CREATE TABLE INDICATORS (IndCode TEXT PRIMARY KEY,IndName TEXT NOT NULL);
CREATE TABLE COUNTRY (CntryCode TEXT PRIMARY KEY,CntryName TEXT NOT NULL,CurrCode TEXT NULL,
                     FOREIGN KEY (CurrCode) REFERENCES CURRENCY(CurrCode));
CREATE TABLE NATIONALACCOUNTS (NatAcctID INTEGER PRIMARY KEY,CntryCode TEXT NOT NULL, Year TEXT,
                               IndCode TEXT,IndValue FLOAT,
                               FOREIGN KEY (CntryCode) REFERENCES COUNTRY (CntryCode)
                              FOREIGN KEY (IndCode) REFERENCES INDICATORS (IndCode));

PRAGMA TABLE_INFO('NATIONALACCOUNTS');


Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


cid,name,type,notnull,dflt_value,pk
0,NatAcctID,INTEGER,0,None,1
1,CntryCode,TEXT,1,None,0
2,Year,TEXT,0,None,0
3,IndCode,TEXT,0,None,0
4,IndValue,FLOAT,0,None,0
